# **Homework 4 - Getting to know your customers**
___

Importing packages

In [ ]:
import pandas as pd
import numpy as np

Cleaning the dataset

In [ ]:
data = pd.read_csv('bank_transactions.csv')

In [ ]:
# We use this function to convert TransactionTime in time
def zeros(x):
    l = len(x)
    if l != 6:
        x = '0' * (6-l) + x
    return x

In [ ]:
# We have about 7000 rows with missing values so we can just drop them
data.dropna(inplace=True)

# Convert in datetime this two columns
data.CustomerDOB = pd.to_datetime(data.CustomerDOB)
data.TransactionDate = pd.to_datetime(data.TransactionDate)

# For TransactionTime we need to split the int in hours, minutes and seconds and convert it in datatime
data.TransactionTime = list(map(str,data.TransactionTime))
data.TransactionTime = list(map(zeros,data.TransactionTime))
data.TransactionTime = pd.to_datetime(data.TransactionTime.apply(lambda x: f'{x[:2]}:{x[2:4]}:{x[4:]}'), format='%H:%M:%S').dt.time

# Adjusting CustomerDOB bringing all the data after 2000 in the previous century and delating all the ones of the 1800
data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)
data.drop(data[data.CustomerDOB.dt.year == 1800].index, axis=0, inplace=True)

# Converting TransactionAmount in integer
data['TransactionAmount (INR)'] = data['TransactionAmount (INR)'].astype(int)

# Saving the new dataset in pickle format
data.to_pickle('clear_dataset.pkl')

___
### **1. Finding Similar Costumers**

#### 1.1 Set up the data

#### 1.2 Fingerprint hashing

#### 1.3 Locality Sensitive Hashing

___
### **2. Grouping customers together!**

#### 2.1 Getting your data + feature engineering

#### 2.2 Choose your features (variables)!

#### 2.3 Clustering!

#### 2.4 Analysing your results!

___
### **Bonus**

___
### **Command Line Question** 

___
### **Algorithmic Question**

In [1]:
import numpy as np

class BColors:
    WARNING = '\033[93m\n[Warning]: '
    ERROR = '\033[91m\n[Error]: '
    ENDC = '\033[0m'

In [4]:
# A function that returns if the given inputs meet the rules of the problem or not.
def flowcheck(entrances, guards, studentsGate):
    # Having an array with the same size of the entrances which turns 1 for each door if open and 0 otherwise.
    entranceState = np.zeros(entrances)
    # A flag variable is defined to be raised in case of a change in the gates
    flag = 0

    for i, j in enumerate(studentsGate):
        # Open the gate
        entranceState[j-1] = 1
        # Check if this gate is the same as the previous one
        if flag != j:
            # Check if the number of guards is greater than the number of opened doors.
            if np.sum(entranceState) > guards:
                return "\nNO"
            # If not, change the flag
            flag = j

        # Check if this door will be used in the future. If not, close the door and release the guard.
        if j not in studentsGate[i+1:]:
            entranceState[j - 1] = 0

    return "\nYES"

In [5]:
# Below we can enter the parameters to test the algorithm. Some input control mechanisms are implemented.
for i in range(1):
    try:
        entrances, students, guards = [int(i) for i in input("\nParameters:\n").split()]
    except:
        print(f"{BColors.ERROR}The input parameters must be space seperated positive integers."
              f"\nPlease try again.{BColors.ENDC}")
        continue

    try:
        studentsGate = [int(i) for i in input("\nEnter each students' gate:\n").split(" ")]
    except:
        print(f"{BColors.ERROR}Students' gates must be written in an array of integers."
              f"\nPlease try again.{BColors.ENDC}")
        continue

    if len(studentsGate) != students:
        print(f"{BColors.ERROR}The length of the entrace array must be equal to the number of students."
              f"\nPlease try again.{BColors.ENDC}")

    if np.max(studentsGate) > entrances:
        print(f"{BColors.ERROR}There are only {entrances} gates.\nPlease try again.{BColors.ENDC}")

    if len(np.array(studentsGate)[np.array(studentsGate)<0]) or entrances < 0 or students < 0 or guards < 0:
        print(f"{BColors.ERROR}All inputs must be positive integers.\nPlease try again.{BColors.ENDC}")

    else:
        print(flowcheck(entrances, guards, studentsGate))
        break


Parameters:
3 6 2

Enter each students' gate:
1 2 2 3 2 1

NO
